In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing libraries

In [ ]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

     |████████████████████████████████| 358kB 11.2MB/s 
     |████████████████████████████████| 20.1MB 1.7MB/s 
     |████████████████████████████████| 2.1MB 36.7MB/s 
     |████████████████████████████████| 225kB 40.3MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 337kB 40.2MB/s 
     |████████████████████████████████| 778kB 36.4MB/s 
     |████████████████████████████████| 901kB 20.2MB/s 
     |████████████████████████████████| 3.3MB 25.2MB/s 
     |████████████████████████████████| 112kB 40.8MB/s 
     |████████████████████████████████| 245kB 43.5MB/s 
     |████████████████████████████████| 1.2MB 35.6MB/s 
     |████████████████████████████████| 19.7MB 1.5MB/s 
     |████████████████████████████████| 983kB 20.8MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 798kB 32.8MB/s 
  Created wheel for lru-dict: filename=lru_dict-1.1.

     |████████████████████████████████| 23.9MB 130kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149800 sha256=7d90d910a6fb9f5a65ab96bd928b269477d63702515a5ee0dfc66d5795f14a35
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import Phrases
from gensim.test.utils import get_tmpfile
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.transformations import WordSwap
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:20<00:00, 23.6MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpashqbyyg.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [ ]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [ ]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [ ]:
if 'cat' in model3.key_to_index:
  print('present')

present


# bert Base Cased-imdb

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-imdb")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Framing fastAttack

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [ ]:
def attack_stats(attack, dataset, num_of_examples=25):
  print(attack)
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

### FastText, GreedySearch, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset imdb, split test.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b. Subsequent calls will reuse this data.


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:07<01:04,  7.20s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:25<01:42, 12.80s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  30%|███       | 3/10 [06:10<14:24, 123.47s/it]

...


[Succeeded / Failed / Total] 8 / 0 / 9:  90%|█████████ | 9/10 [08:42<00:58, 58.01s/it]


[Succeeded / Failed / Total] 9 / 0 / 10: 100%|██████████| 10/10 [11:46<00:00, 70.67s/it]


[Succeeded / Failed / Total] 9 / 0 / 10: : 11it [12:45, 69.57s/it]                      
[Succeeded / Failed / Total] 10 / 0 / 11: : 11it [12:45, 69.57s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 5.64%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 1633.6 |
+-------------------------------+--------+


textattack: Attack time: 765.2902481555939s


# Framing word2vec attack

In [ ]:
class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

### Word2vec, GreedySearch, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:24<03:41, 24.65s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:37<02:30, 18.76s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  30%|███       | 3/10 [01:07<02:36, 22.34s/it]

...


[Succeeded / Failed / Total] 8 / 0 / 9:  90%|█████████ | 9/10 [04:53<00:32, 32.61s/it]


[Succeeded / Failed / Total] 9 / 0 / 10: 100%|██████████| 10/10 [10:28<00:00, 62.82s/it]


[Succeeded / Failed / Total] 9 / 0 / 10: : 11it [10:49, 59.02s/it]                      
[Succeeded / Failed / Total] 10 / 0 / 11: : 11it [10:49, 59.02s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 5.31%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 1370.0 |
+-------------------------------+--------+


textattack: Attack time: 649.2144260406494s


### Framing Glove attack

In [ ]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

### Glove, GreedySearch, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:25<03:50, 25.59s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:45<03:02, 22.76s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  30%|███       | 3/10 [02:48<06:33, 56.21s/it]

...


[Succeeded / Failed / Total] 7 / 1 / 9:  90%|█████████ | 9/10 [08:01<00:53, 53.50s/it]


[Succeeded / Failed / Total] 8 / 1 / 10: 100%|██████████| 10/10 [12:33<00:00, 75.34s/it]


[Succeeded / Failed / Total] 8 / 1 / 10: : 11it [13:45, 75.05s/it]                      
[Succeeded / Failed / Total] 9 / 1 / 11: : 11it [13:45, 75.05s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 9.09%  |
| Attack success rate:          | 90.0%  |
| Average perturbed word %:     | 3.7%   |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 1202.8 |
+-------------------------------+--------+


textattack: Attack time: 825.5916497707367s


### FastText, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:04<00:44,  4.89s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:16<01:06,  8.34s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [00:32<01:16, 10.91s/it]

...


[Succeeded / Failed / Total] 6 / 2 / 9:  90%|█████████ | 9/10 [01:19<00:08,  8.85s/it]


[Succeeded / Failed / Total] 7 / 2 / 10: 100%|██████████| 10/10 [01:58<00:00, 11.90s/it]


[Succeeded / Failed / Total] 7 / 2 / 10: : 11it [02:15, 12.30s/it]                      
[Succeeded / Failed / Total] 7 / 3 / 11: : 11it [02:15, 12.30s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 27.27% |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 8.56%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 347.4  |
+-------------------------------+--------+


textattack: Attack time: 135.31681871414185s


### Word2Vec, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:  10%|█         | 1/10 [00:08<01:12,  8.06s/it]


[Succeeded / Failed / Total] 1 / 1 / 2:  20%|██        | 2/10 [00:20<01:20, 10.04s/it]


[Succeeded / Failed / Total] 1 / 2 / 3:  30%|███       | 3/10 [00:33<01:19, 11.30s/it]

...


[Succeeded / Failed / Total] 4 / 4 / 9:  90%|█████████ | 9/10 [01:34<00:10, 10.48s/it]


[Succeeded / Failed / Total] 5 / 4 / 10: 100%|██████████| 10/10 [02:35<00:00, 15.50s/it]


[Succeeded / Failed / Total] 5 / 4 / 10: : 11it [02:49, 15.39s/it]                      
[Succeeded / Failed / Total] 5 / 5 / 11: : 11it [02:49, 15.39s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 45.45% |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 11.07% |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 395.4  |
+-------------------------------+--------+


textattack: Attack time: 169.28298711776733s


### Glove, GreedyWordSwapWIR RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:06<00:54,  6.03s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:18<01:15,  9.48s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [00:36<01:24, 12.05s/it]

...


[Succeeded / Failed / Total] 5 / 3 / 9:  90%|█████████ | 9/10 [01:35<00:10, 10.56s/it]


[Succeeded / Failed / Total] 6 / 3 / 10: 100%|██████████| 10/10 [02:06<00:00, 12.70s/it]


[Succeeded / Failed / Total] 6 / 3 / 10: : 11it [02:19, 12.67s/it]                      
[Succeeded / Failed / Total] 7 / 3 / 11: : 11it [02:19, 12.67s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 27.27% |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 7.38%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 335.6  |
+-------------------------------+--------+


textattack: Attack time: 139.39254665374756s


### FastText, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [ ]:
from textattack.search_methods import ParticleSwarmOptimization


# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:39<05:59, 39.91s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [02:35<10:23, 77.98s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [08:05<18:53, 161.99s/it]

...


[Succeeded / Failed / Total] 6 / 2 / 9:  90%|█████████ | 9/10 [19:29<02:09, 129.96s/it]


[Succeeded / Failed / Total] 7 / 2 / 10: 100%|██████████| 10/10 [37:24<00:00, 224.43s/it]


[Succeeded / Failed / Total] 7 / 2 / 10: : 11it [43:32, 237.52s/it]                      
[Succeeded / Failed / Total] 7 / 3 / 11: : 11it [43:32, 237.52s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 27.27% |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 2.49%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 4875.2 |
+-------------------------------+--------+


textattack: Attack time: 2612.7634756565094s


### Word2Vec, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [02:05<18:46, 125.16s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [03:02<12:09, 91.16s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  30%|███       | 3/10 [06:28<15:06, 129.55s/it]

...


[Succeeded / Failed / Total] 6 / 2 / 9:  90%|█████████ | 9/10 [20:32<02:16, 136.99s/it]


[Succeeded / Failed / Total] 6 / 3 / 10: 100%|██████████| 10/10 [41:25<00:00, 248.51s/it]


[Succeeded / Failed / Total] 6 / 3 / 10: : 11it [42:34, 232.24s/it]                      
[Succeeded / Failed / Total] 7 / 3 / 11: : 11it [42:34, 232.24s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 27.27% |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 5.12%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 5015.4 |
+-------------------------------+--------+


textattack: Attack time: 2554.6651067733765s


### Glove, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [02:35<23:16, 155.19s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [03:56<15:44, 118.11s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [10:10<23:44, 203.55s/it]

...


[Succeeded / Failed / Total] 5 / 3 / 9:  90%|█████████ | 9/10 [26:02<02:53, 173.62s/it]


[Succeeded / Failed / Total] 6 / 3 / 10: 100%|██████████| 10/10 [36:56<00:00, 221.65s/it]


[Succeeded / Failed / Total] 6 / 3 / 10: : 11it [42:42, 232.98s/it]                      
[Succeeded / Failed / Total] 6 / 4 / 11: : 11it [42:42, 232.98s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 36.36% |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 3.69%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 3849.0 |
+-------------------------------+--------+


textattack: Attack time: 2562.838896751404s


### FastText, GreedyWordSwapWIR, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)
dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset imdb, split test.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b. Subsequent calls will reuse this data.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:03<00:28,  3.18s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:09<00:38,  4.80s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [00:19<00:45,  6.51s/it]

...

[Succeeded / Failed / Total] 6 / 2 / 9:  90%|█████████ | 9/10 [01:06<00:07,  7.42s/it]

[Succeeded / Failed / Total] 7 / 2 / 10: 100%|██████████| 10/10 [01:19<00:00,  7.97s/it]

[Succeeded / Failed / Total] 7 / 3 / 11: : 11it [01:29,  8.17s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 27.27% |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 12.57% |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 379.0  |
+-------------------------------+--------+


textattack: Attack time: 89.83300042152405s


### Word2Vec, GreedyWordSwapWIR, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:  10%|█         | 1/10 [00:05<00:46,  5.17s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:  20%|██        | 2/10 [00:11<00:47,  5.98s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  30%|███       | 3/10 [00:21<00:49,  7.02s/it]

...

[Succeeded / Failed / Total] 4 / 4 / 9:  90%|█████████ | 9/10 [01:00<00:06,  6.67s/it]

[Succeeded / Failed / Total] 5 / 4 / 10: 100%|██████████| 10/10 [01:49<00:00, 10.92s/it]

[Succeeded / Failed / Total] 5 / 5 / 11: : 11it [01:58, 10.77s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 45.45% |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 11.68% |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 421.5  |
+-------------------------------+--------+


textattack: Attack time: 118.48360466957092s


### Glove, GreedyWordSwapWIR, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:03<00:27,  3.06s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:10<00:40,  5.09s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [00:22<00:53,  7.63s/it]

...

[Succeeded / Failed / Total] 5 / 3 / 9:  90%|█████████ | 9/10 [01:10<00:07,  7.78s/it]

[Succeeded / Failed / Total] 6 / 3 / 10: 100%|██████████| 10/10 [02:04<00:00, 12.42s/it]

[Succeeded / Failed / Total] 7 / 3 / 11: : 11it [02:17, 12.48s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 27.27% |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 9.58%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 371.9  |
+-------------------------------+--------+


textattack: Attack time: 137.25889444351196s


### FastText, ParticleSwarmOptimization, PartsofSpeech

In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:15<02:16, 15.15s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [01:15<05:00, 37.56s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [05:42<13:19, 114.20s/it]

...

[Succeeded / Failed / Total] 5 / 3 / 9:  90%|█████████ | 9/10 [17:43<01:58, 118.16s/it]

[Succeeded / Failed / Total] 6 / 3 / 10: 100%|██████████| 10/10 [36:28<00:00, 218.84s/it]

[Succeeded / Failed / Total] 6 / 4 / 11: : 11it [40:28, 220.74s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 36.36% |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 2.77%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 6954.3 |
+-------------------------------+--------+


textattack: Attack time: 2428.137699365616s


### Word2Vec, ParticleSwarmOptimization, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:  10%|█         | 1/10 [01:56<17:24, 116.06s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:  20%|██        | 2/10 [02:35<10:21, 77.72s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [04:47<11:09, 95.71s/it]

...

[Succeeded / Failed / Total] 4 / 4 / 9:  90%|█████████ | 9/10 [15:47<01:45, 105.30s/it]

[Succeeded / Failed / Total] 5 / 4 / 10: 100%|██████████| 10/10 [36:42<00:00, 220.21s/it]

[Succeeded / Failed / Total] 6 / 4 / 11: : 11it [37:45, 205.97s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 36.36% |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 3.36%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 6263.5 |
+-------------------------------+--------+


textattack: Attack time: 2265.7094497680664s


### Glove, ParticleSwarmOptimization, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b. Subsequent calls will reuse this data.


textattack: Loading datasets dataset imdb, split test.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [01:56<17:30, 116.77s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [04:10<16:42, 125.28s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  30%|███       | 3/10 [15:21<35:50, 307.19s/it]

...

[Succeeded / Failed / Total] 3 / 5 / 9:  90%|█████████ | 9/10 [46:18<05:08, 308.71s/it]

[Succeeded / Failed / Total] 4 / 5 / 10: 100%|██████████| 10/10 [1:18:17<00:00, 469.78s/it]

[Succeeded / Failed / Total] 5 / 5 / 11: : 11it [1:25:20, 465.48s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.91% |
| Accuracy under attack:        | 45.45% |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 1.94%  |
| Average num. words per input: | 295.18 |
| Avg num queries:              | 4584.2 |
+-------------------------------+--------+


textattack: Attack time: 5120.3273713588715s
